# PipelineEndpoint Demo

In [1]:
from azureml.pipeline.wrapper import Module, dsl
from workspace_helpers import setup_default_workspace, load_or_register_dataset
from azureml.pipeline.wrapper._pipeline_endpoint import PipelineEndpoint

In [33]:
workspace = setup_default_workspace()

In [3]:
experiment_name = "TestPipelineEndpoint"
module_func = Module.load(workspace, namespace='azureml', name='Select Columns in Dataset')
test_pipeline_endpoint_name = "test_pipeline_endpoint"
test_pipeline_endpoint_id = "b27eb28c-da34-4064-9c13-e515fdb2c71b"

blob_input_data = load_or_register_dataset(workspace, name='Automobile_price_data_(Raw)',
                                           datastore_name='azureml_globaldatasets',
                                           path='GenericCSV/Automobile_price_data_(Raw)')
select_columns = "{\"isFilter\":true,\"rules\":[{\"exclude\":true,\"ruleType\":\"ColumnNames\",\"columns\":" \
                 "[\"normalized-losses\"]}]}"

@dsl.pipeline(name='test-pipeline', default_compute_target='aml-compute')
def test_pipeline(_select_columns):
    module1 = module_func(
        dataset=blob_input_data,
        select_columns=_select_columns
    )
    return module1.outputs

In [4]:
pipeline = test_pipeline(_select_columns=select_columns)
pipeline_run = pipeline.submit(experiment_name=experiment_name)

Submitted PipelineRun 4c584969-a9ac-410a-bb01-dd1c23075163
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/TestPipelineEndpoint/runs/4c584969-a9ac-410a-bb01-dd1c23075163?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS


### 1. Publish
Create a PipelineEndpoint with the specified name and pipeline/pipeline_run.

#### 1.1 publish by pipeline

In [5]:
pipeline_endpoint_published_by_pipeline = PipelineEndpoint.publish(
    workspace=workspace,
    name=test_pipeline_endpoint_name,
    description="description_for_test_endpoint_publish",
    pipeline=pipeline)

pipeline_endpoint_published_by_pipeline

Using existing pipeline endpoint "test_pipeline_endpoint"
Pipeline endpoint with name "test_pipeline_endpoint" already exist, published pipeline to "test_pipeline_endpoint"


Name,Description,Date updated,Updated by,Last run submit time,Last run status,Status,Portal Link
test_pipeline_endpoint,,2020-08-11 08:33:13.271820+00:00,Liyuan Wu,2020-08-11 08:11:48.819178+00:00,2,Active,Link


#### 1.2 publish by pipeline_run

In [6]:
# pipeline_run test
pipeline_endpoint_published_by_pipeline_run = PipelineEndpoint.publish(
    workspace=workspace,
    name=test_pipeline_endpoint_name,
    description="description_for_test_endpoint_publish",
    pipeline=pipeline_run
)

pipeline_endpoint_published_by_pipeline_run

Using existing pipeline endpoint "test_pipeline_endpoint"
Pipeline endpoint with name "test_pipeline_endpoint" already exist, published pipeline to "test_pipeline_endpoint"


Name,Description,Date updated,Updated by,Last run submit time,Last run status,Status,Portal Link
test_pipeline_endpoint,,2020-08-11 08:33:33.722210+00:00,Liyuan Wu,2020-08-11 08:11:48.819178+00:00,2,Active,Link


### 2. Get
Get the PipelineEndpoint by name or ID

#### 2.1 get by id

In [7]:
endpoint_got_by_id = PipelineEndpoint.get(workspace=workspace, id=test_pipeline_endpoint_id)

endpoint_got_by_id

Name,Description,Date updated,Updated by,Last run submit time,Last run status,Status,Portal Link
test_pipeline_endpoint,,2020-08-11 08:33:33.722210+00:00,Liyuan Wu,2020-08-11 08:11:48.819178+00:00,2,Active,Link


#### 2.2 get by name

In [8]:
endpoint_got_by_name = PipelineEndpoint.get(workspace=workspace, name=test_pipeline_endpoint_name)

endpoint_got_by_name


Name,Description,Date updated,Updated by,Last run submit time,Last run status,Status,Portal Link
test_pipeline_endpoint,,2020-08-11 08:33:33.722210+00:00,Liyuan Wu,2020-08-11 08:11:48.819178+00:00,2,Active,Link


In [9]:
endpoint_got_by_name._get_portal_url()

'https://ml.azure.com/endpoint/b27eb28c-da34-4064-9c13-e515fdb2c71b/test_pipeline_endpoint/pipelines?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS'

### 3. Submit
Submit default version of endpoint.

In [10]:
test_submit_experiment_name = "test_submit_by_pipeline_endpoint"
pipeline_endpoint = PipelineEndpoint.get(workspace=workspace, id=test_pipeline_endpoint_id)

paramesters = {'_select_columns': '{"isFilter":true,"rules":[{"exclude":true,"ruleType":"ColumnNames","columns":["normalized-losses"]}]}'}

run_submitted_by_pipeline_endpoint = pipeline_endpoint.submit(experiment_name=test_submit_experiment_name,
                                                              parameters=paramesters)

Submitted PipelineEndpointRun a30e8502-d1e0-48af-8bf7-06ba3317b5f7
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/test_submit_by_pipeline_endpoint/runs/a30e8502-d1e0-48af-8bf7-06ba3317b5f7?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS


In [11]:
run_submitted_by_pipeline_endpoint.tags["azureml.pipelineid"]

'315469be-6c26-404b-9844-eae0f6757756'

In [12]:
pipeline_endpoint._default_pipeline_id

'315469be-6c26-404b-9844-eae0f6757756'

In [13]:
run_submitted_by_pipeline_endpoint

Experiment,Id,Type,Status,Details Page,Docs Page
test_submit_by_pipeline_endpoint,a30e8502-d1e0-48af-8bf7-06ba3317b5f7,azureml.PipelineRun,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


### 4. List
List active PipelineEndpoints in the current workspace.

In [14]:
pipeline_endpoints_list = PipelineEndpoint.list(workspace=workspace, active_only=True)

In [15]:
pipeline_endpoints_list

[PipelineEndpoint(Name: test_pipeline_endpoint,
 Description: None,
 Date updated: 2020-08-11 08:33:33.722210+00:00,
 Updated by: Liyuan Wu,
 Last run submit time: 2020-08-11 08:34:09.957403+00:00,
 Last run status: 1,
 Status: Active,
 Portal Link: https://ml.azure.com/endpoint/b27eb28c-da34-4064-9c13-e515fdb2c71b/test_pipeline_endpoint/pipelines?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS),
 PipelineEndpoint(Name: test_published_pipeline,
 Description: None,
 Date updated: 2020-08-11 08:34:12.886690+00:00,
 Updated by: 240bbec1-db0f-4c9c-a103-de1dd781f3f2,
 Last run submit time: 2020-08-11 08:30:53.251555+00:00,
 Last run status: 3,
 Status: Active,
 Portal Link: https://ml.azure.com/endpoint/4ba503fa-ee2c-4200-a28e-4a96134b4dba/test_published_pipeline/pipelines?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS),
 PipelineEndpoint(Name: test_publish_

In [16]:
print("pipeline endpoints name list:")
[endpoint.name for endpoint in pipeline_endpoints_list]

pipeline endpoints name list:


['test_pipeline_endpoint',
 'test_published_pipeline',
 'test_publish_to_pipeline',
 'test_endpoint_publish_endpoint',
 'test_endpoint_publish_endpoint_new',
 'A huge pipeline composed with nodes 1 in 2 outs',
 'pipeline_export_test',
 'test',
 'test-sub-graph',
 'Pipeline-Created-on-7-6-2020111',
 'Pipeline-Created-on-7-6-2020',
 'Sample 3: test2',
 'Sample 3: Binary Classification with Feature Selection - Income Prediction',
 'Sample 1: Regression - Automobile Price Prediction (Basic)']

### 5. Enable & Disable

In [17]:
pipeline_endpoint.enable()
print(pipeline_endpoint.status)

Active


In [18]:
pipeline_endpoint.disable()
print(pipeline_endpoint.status)

Disabled


In [19]:
pipeline_endpoint.enable()
print(pipeline_endpoint.status)

Active


### 6. Get Pipeline
Get the pipeline of a specified version or the default; throws an exception if the version is not found.

In [20]:
pipeline_version = '0'
pipeline_got = pipeline_endpoint.get_pipeline(version=pipeline_version)

In [21]:
pipeline_got

Name,Description,Date updated,Published by,Last run submit time,Last run status,Status,Portal Link
test_pipeline_endpoint,,2020-08-03 04:21:50.112827+00:00,Brynn Yin,,0,Disabled,Link


In [22]:
# no version is specified, got the default version
pipeline_default = pipeline_endpoint.get_pipeline()

print('pipeline_default.id:', pipeline_default.id)
print('pipeline_endpoint default version:', pipeline_endpoint._default_pipeline_id)

pipeline_default.id: 315469be-6c26-404b-9844-eae0f6757756
pipeline_endpoint default version: 315469be-6c26-404b-9844-eae0f6757756


### 7. Get pipelines
Get list of PublishedPipelines in PipelineEndpoint, with format {"version": version, "pipeline": PublihsedPipeline}

active_only: Whether to return only active pipelines.

In [23]:
pipelines_list = pipeline_endpoint.get_pipelines(active_only=True)

In [24]:
pipelines_list

{'278': PublishedPipeline(Name: TestPipelineEndpoint-08-11-2020-16-33-published,
 Description: TestPipelineEndpointdescription for TestPipelineEndpoint-08-11-2020-16-33-published,
 Date updated: 2020-08-11 08:33:33.722989+00:00,
 Published by: Liyuan Wu,
 Last run submit time: 2020-08-11 08:34:09.957403+00:00,
 Last run status: 2,
 Status: Active,
 Portal Link: https://ml.azure.com/pipeline/published/315469be-6c26-404b-9844-eae0f6757756/detail?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS),
 '277': PublishedPipeline(Name: test-pipeline-08-11-2020-16-33-published,
 Description: description for test-pipeline-08-11-2020-16-33-published,
 Date updated: 2020-08-11 08:33:13.258049+00:00,
 Published by: Liyuan Wu,
 Last run submit time: None,
 Last run status: 0,
 Status: Active,
 Portal Link: https://ml.azure.com/pipeline/published/53d58e0e-bf94-41ff-97fd-40514115cbe4/detail?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197

### 8. Set default
Set the default version of PipelineEndpoint, throws an exception if the specified pipeline is not found.

In [25]:
default_version = '0'
default_pipeline = pipeline_endpoint.get_pipeline(version=default_version)
print('default version BEFORE set default version:', pipeline_endpoint.default_version)
pipeline_endpoint.set_default(pipeline=default_pipeline)
print('default version AFTER set default version:', pipeline_endpoint.default_version)

default version BEFORE set default version: 278
default version AFTER set default version: 0


### 9. Set default version

In [26]:
default_version = '0'
print('default version BEFORE set default version:', pipeline_endpoint.default_version)
pipeline_endpoint.set_default_version(version=default_version)
print('default version AFTER set default version:', pipeline_endpoint.default_version)

default version BEFORE set default version: 0
default version AFTER set default version: 0


# Finish

In [31]:
version_list = list(pipelines_list.keys())
pipelines_list[version_list[0]].disable()
pipelines_list[version_list[1]].disable()